In [7]:
import tensorflow as tf
import tensorflow_hub as hub

CHANNELS = 3 # number of image channels (RGB)

def build_graph(hub_module_url, target_image_path):
  # Step 1) Prepare pre-trained model for extracting image features.
  module = hub.Module(hub_module_url)
  height, width = hub.get_expected_image_size(module)

  # Copied a method of https://github.com/GoogleCloudPlatform/cloudml-samples/blob/bf0680726/flowers/trainer/model.py#L181
  # and fixed for all type images (not only jpeg)
  def decode_and_resize(image_str_tensor):
    """Decodes jpeg string, resizes it and returns a uint8 tensor."""
    image = tf.image.decode_image(image_str_tensor, channels=CHANNELS)
    # Note resize expects a batch_size, but tf_map supresses that index,
    # thus we have to expand then squeeze.  Resize returns float32 in the
    # range [0, uint8_max]
    image = tf.expand_dims(image, 0)
    image = tf.image.resize_bilinear(
        image, [height, width], align_corners=False)
    image = tf.squeeze(image, squeeze_dims=[0])
    image = tf.cast(image, dtype=tf.uint8)
    return image

  def to_img_feature(images):
    """Extract the feature of image vectors"""
    outputs = module(dict(images=images), signature="image_feature_vector", as_dict=True)
    return outputs['default']

  # Step 2) Extract image features of the target image.
  target_image_bytes = tf.gfile.GFile(target_image_path, 'rb').read()
  target_image = tf.constant(target_image_bytes, dtype=tf.string)
  target_image = decode_and_resize(target_image)
  target_image = tf.image.convert_image_dtype(target_image, dtype=tf.float32)
  target_image = tf.expand_dims(target_image, 0)
  target_image = to_img_feature(target_image)

  # Step 3) Extract image features of input images.
  input_byte = tf.placeholder(tf.string, shape=[None])
  input_image = tf.map_fn(decode_and_resize, input_byte, back_prop=False, dtype=tf.uint8)
  input_image = tf.image.convert_image_dtype(input_image, dtype=tf.float32)
  input_image = to_img_feature(input_image)

  # Step 4) Compare cosine_similarities of the target image and the input images.
  dot = tf.tensordot(target_image, tf.transpose(input_image), 1)
  similarity = dot / (tf.norm(target_image, axis=1) * tf.norm(input_image, axis=1))
  similarity = tf.reshape(similarity, [-1])
  
  return input_byte, similarity

In [8]:
target_image_url = "https://d2192bm55jmxp1.cloudfront.net/resize/s/profile/201712/B4CFF0B690BB910AF5CB97A2122E9AEC427DAD810E4E0382B13643C9A8D22A55.jpg" #@param {type:"string"}
input_image1_url = "https://d2192bm55jmxp1.cloudfront.net/resize/s/profile/201706/5b193314eff5f5b90faf9e5600d2c95c0c86c4c154414ea8a69bc26942a37613.jpg" #@param {type:"string"}
input_image2_url = "https://d2192bm55jmxp1.cloudfront.net/resize/s/profile/201803/b15cd493adfd367b6663786ca21c4ecdeba67c859ac628849ebd4c3f7990f92b.jpg" #@param {type:"string"}
input_image3_url = "https://d2192bm55jmxp1.cloudfront.net/resize/s/profile/201804/8d4b782228f43fb859076c41ae69a1920aec1b20154ea059a8e87142b66a9841.jpg" #@param {type:"string"}
input_image4_url = "https://d2192bm55jmxp1.cloudfront.net/resize/s/profile/201803/4D2C44F3BB10CB84D0D6894321ED9C8761BB364786550762DC13B125B4C34361.jpg" #@param {type:"string"}

input_image_urls = [input_image1_url, input_image2_url, input_image3_url, input_image4_url]

target_img_path = 'target_img.jpg'
input_img_paths = []

!wget -q {target_image_url} -O {target_img_path}

for i, url in enumerate(input_image_urls):
  if len(url) > 0:
    path = "input_img%d.jpg" % i
    !wget -q {url} -O {path}
    input_img_paths.append(path)

'wget'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'wget'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'wget'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'wget'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'wget'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [9]:
import time
import tensorflow as tf
from IPython.display import Image, display
tf.logging.set_verbosity(tf.logging.ERROR)

# Load bytes of image files
image_bytes = [tf.gfile.GFile(name, 'rb').read()
               for name in [target_img_path] + input_img_paths]

hub_module_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_96/feature_vector/1" #@param {type:"string"}

with tf.Graph().as_default():
  input_byte, similarity_op = build_graph(hub_module_url, target_img_path)
  
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    t0 = time.time() # for time check
    
    # Inference similarities
    similarities = sess.run(similarity_op, feed_dict={input_byte: image_bytes})
    
    print("%d images inference time: %.2f s" % (len(similarities), time.time() - t0))

# Display results
print("# Target image")
display(Image(target_img_path))
print("- similarity: %.2f" % similarities[0])

print("# Input images")
for similarity, input_img_path in zip(similarities[1:], input_img_paths):
  display(Image(input_img_path))
  print("- similarity: %.2f" % similarity)

AttributeError: module 'tensorflow' has no attribute 'logging'